# Getting Dataset

In [ ]:
pip install kagglehub


In [5]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("kaushiksuresh147/bitcoin-tweets")

print("Path to dataset files:", path)

c:\Users\Zunaira\Documents\8th Semester\DDE\A01\Distributed-Data-Engineering-Bitcoin-Tweets-using-DynamoDB\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 695M/695M [08:34<00:00, 1.42MB/s] 

Extracting files...


Path to dataset files: C:\Users\Zunaira\.cache\kagglehub\datasets\kaushiksuresh147\bitcoin-tweets\versions\54


## Analyzing Dataset

In [39]:
import pandas as pd

# Read the first 500 rows of the CSV file
df = pd.read_csv('data/Bitcoin_tweets.csv', nrows=500)  # Replace 'your_file.csv' with your actual file path

# Display basic info about the dataframe
print("=== DataFrame Info ===")
print(df.info())

# Display the data types of each column
print("\n=== Column Data Types ===")
print(df.dtypes)

# Display sample data from each column to inspect values
print("\n=== Sample Data (First 5 Rows) ===")
print(df.head())

# For numeric columns that might have issues, show value counts
numeric_cols = ['user_followers', 'user_friends', 'user_favourites']
for col in numeric_cols:
    if col in df.columns:
        print(f"\n=== Value counts for {col} ===")
        print(df[col].value_counts(dropna=False).head(20))  # Show top 20 values including NaN

=== DataFrame Info ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   user_name         500 non-null    object 
 1   user_location     303 non-null    object 
 2   user_description  480 non-null    object 
 3   user_created      500 non-null    object 
 4   user_followers    500 non-null    float64
 5   user_friends      500 non-null    int64  
 6   user_favourites   500 non-null    int64  
 7   user_verified     500 non-null    bool   
 8   date              500 non-null    object 
 9   text              500 non-null    object 
 10  hashtags          405 non-null    object 
 11  source            496 non-null    object 
 12  is_retweet        500 non-null    bool   
dtypes: bool(2), float64(1), int64(2), object(8)
memory usage: 44.1+ KB
None

=== Column Data Types ===
user_name            object
user_location        object
user_de

In [16]:
import boto3
import pandas as pd
import json
from datetime import datetime
from boto3.dynamodb.conditions import Key

In [34]:
# Explicit credentials and session
session = boto3.Session(
    aws_access_key_id='dummy',
    aws_secret_access_key='dummy',
    region_name='us-east-1'
)

In [ ]:
# Use session for both resource and client
dynamodb = session.resource(
    'dynamodb',
    endpoint_url='http://localhost:8000'
)

In [36]:
table = dynamodb.Table('BitcoinTweets')

## Create Table

In [17]:

def create_table():
    # Create the table if it doesn't exist
    table = dynamodb.create_table(
        TableName='BitcoinTweets',
        KeySchema=[
            {'AttributeName': 'user_name', 'KeyType': 'HASH'},  # Partition key
            {'AttributeName': 'date', 'KeyType': 'RANGE'}       # Sort key
        ],
        AttributeDefinitions=[
            {'AttributeName': 'user_name', 'AttributeType': 'S'},
            {'AttributeName': 'date', 'AttributeType': 'S'},
            {'AttributeName': 'user_location', 'AttributeType': 'S'},
            {'AttributeName': 'user_followers', 'AttributeType': 'N'},
            {'AttributeName': 'dummy_attribute', 'AttributeType': 'S'},
            {'AttributeName': 'hashtag', 'AttributeType': 'S'}
        ],
        GlobalSecondaryIndexes=[
            {
                'IndexName': 'LocationDateIndex',
                'KeySchema': [
                    {'AttributeName': 'user_location', 'KeyType': 'HASH'},
                    {'AttributeName': 'date', 'KeyType': 'RANGE'}
                ],
                'Projection': {'ProjectionType': 'ALL'},
                'ProvisionedThroughput': {'ReadCapacityUnits': 5, 'WriteCapacityUnits': 5}
            },
            {
                'IndexName': 'FollowersIndex',
                'KeySchema': [
                    {'AttributeName': 'dummy_attribute', 'KeyType': 'HASH'},
                    {'AttributeName': 'user_followers', 'KeyType': 'RANGE'}
                ],
                'Projection': {'ProjectionType': 'ALL'},
                'ProvisionedThroughput': {'ReadCapacityUnits': 5, 'WriteCapacityUnits': 5}
            },
            {
                'IndexName': 'HashtagIndex',
                'KeySchema': [
                    {'AttributeName': 'hashtag', 'KeyType': 'HASH'},
                    {'AttributeName': 'date', 'KeyType': 'RANGE'}
                ],
                'Projection': {'ProjectionType': 'ALL'},
                'ProvisionedThroughput': {'ReadCapacityUnits': 5, 'WriteCapacityUnits': 5}
            }
        ],
        ProvisionedThroughput={'ReadCapacityUnits': 5, 'WriteCapacityUnits': 5}
    )
    return table

## Insert Data

In [50]:
def insert_data(csv_file):
    dynamodb = boto3.resource('dynamodb', region_name='us-east-1')  # or your preferred region

    table = dynamodb.Table('BitcoinTweets')
    
    chunksize = 500
    success_count = 0
    error_count = 0

    def safe_convert_to_int(value):
        """Safely convert various string formats to integers"""
        if pd.isna(value):
            return 0
        try:
            # Handle 'False' strings
            if str(value).lower() == 'false':
                return 0
            # Handle float strings like '425.0'
            if isinstance(value, str) and '.' in value:
                return int(float(value))
            return int(value)
        except:
            return 0

    for chunk in pd.read_csv(csv_file, chunksize=chunksize):
        with table.batch_writer(overwrite_by_pkeys=['user_name', 'date']) as batch:
            for _, row in chunk.iterrows():
                try:
                    # Process hashtags
                    hashtags = []
                    if pd.notna(row['hashtags']):
                        try:
                            hashtags = eval(row['hashtags']) if isinstance(row['hashtags'], str) else []
                        except:
                            hashtags = str(row['hashtags']).strip('[]').replace("'", "").split(', ')
                    
                    # Convert dates to string format
                    date_str = str(pd.to_datetime(row['date']).date()) if pd.notna(row['date']) else ''
                    user_created = str(pd.to_datetime(row['user_created']).date()) if pd.notna(row['user_created']) else ''

                    item = {
                        'user_name': str(row['user_name']),
                        'date': date_str,
                        'user_location': str(row['user_location']) if pd.notna(row['user_location']) else 'unknown',
                        'user_description': str(row['user_description']) if pd.notna(row['user_description']) else '',
                        'user_created': user_created,
                        'user_followers': safe_convert_to_int(row['user_followers']),
                        'user_friends': safe_convert_to_int(row['user_friends']),
                        'user_favourites': safe_convert_to_int(row['user_favourites']),
                        'user_verified': bool(row['user_verified']),
                        'text': str(row['text']),
                        'hashtags': hashtags,
                        'source': str(row['source']) if pd.notna(row['source']) else '',
                        'is_retweet': bool(row['is_retweet']),
                        'dummy_attribute': 'ALL_USERS'
                    }

                    batch.put_item(Item=item)
                    success_count += 1
                except Exception as e:
                    error_count += 1
                    if error_count < 10:  # Only show first 10 errors
                        print(f"❌ Error in row {_}: {str(e)}")
                        print(f"Problematic values: {row[['user_name', 'date', 'user_followers', 'user_friends', 'user_favourites']].to_dict()}")

    print(f"\n✅ Data insertion completed. Success: {success_count}, Errors: {error_count}")

In [40]:
def insert_data(csv_file):
    chunksize = 500  # Tune if needed

    for chunk in pd.read_csv(csv_file, chunksize=chunksize):
        with table.batch_writer(overwrite_by_pkeys=['user_name', 'date']) as batch:
            for _, row in chunk.iterrows():
                try:
                    # Process hashtags (convert from string representation of list to actual list)
                    hashtags = []
                    if pd.notna(row['hashtags']):
                        try:
                            # Handle both "[...]" and NaN cases
                            hashtags = eval(row['hashtags']) if isinstance(row['hashtags'], str) else []
                        except:
                            # Fallback for malformed hashtag strings
                            hashtags = str(row['hashtags']).strip('[]').replace("'", "").split(', ')
                    
                    # Convert date to string format (assuming it's in ISO format)
                    date_str = str(pd.to_datetime(row['date']).date()) if pd.notna(row['date']) else ''

                    item = {
                        'user_name': str(row['user_name']),
                        'date': date_str,  # Using just the date part
                        'user_location': str(row['user_location']) if pd.notna(row['user_location']) else 'unknown',
                        'user_description': str(row['user_description']) if pd.notna(row['user_description']) else '',
                        'user_created': str(pd.to_datetime(row['user_created']).date()) if pd.notna(row['user_created']) else '',
                        'user_followers': int(float(row['user_followers'])),  # Convert float to int
                        'user_friends': int(row['user_friends']),
                        'user_favourites': int(row['user_favourites']),
                        'user_verified': bool(row['user_verified']),
                        'text': str(row['text']),
                        'hashtags': hashtags,
                        'source': str(row['source']) if pd.notna(row['source']) else '',
                        'is_retweet': bool(row['is_retweet']),
                        'dummy_attribute': 'ALL_USERS'
                    }

                    batch.put_item(Item=item)
                except Exception as e:
                    print(f"❌ Skipping row due to error: {e}")
                    print(f"Problematic row: {row.to_dict()}")  # Show the entire row as dict

    print("✅ Data inserted successfully from Kaggle dataset!")

# cross


In [ ]:
def insert_data(csv_file):
    chunksize = 500  # Tune if needed

    for chunk in pd.read_csv(csv_file, chunksize=chunksize):
        with table.batch_writer(overwrite_by_pkeys=['user_name', 'date']) as batch:
            for _, row in chunk.iterrows():
                try:
                    hashtags = row['hashtags'].strip('[]').replace("'", "").split(', ') if pd.notna(row['hashtags']) else []

                    item = {
                        'user_name': str(row['user_name']),
                        'date': str(row['date']),
                        'user_location': str(row['user_location']) if pd.notna(row['user_location']) else 'unknown',
                        'user_description': str(row['user_description']) if pd.notna(row['user_description']) else '',
                        'user_created': str(row['user_created']),
                        'user_followers': int(row['user_followers']) if pd.notna(row['user_followers']) else 0,
                        'user_friends': int(row['user_friends']) if pd.notna(row['user_friends']) else 0,
                        'user_favourites': int(row['user_favourites']) if 'user_favourites' in row and pd.notna(row['user_favourites']) else 0,
                        'user_verified': bool(row['user_verified']) if pd.notna(row['user_verified']) else False,
                        'text': str(row['text']),
                        'hashtags': hashtags,
                        'source': str(row['source']) if pd.notna(row['source']) else '',
                        'is_retweet': bool(row['is_retweet']) if pd.notna(row['is_retweet']) else False,
                        'dummy_attribute': 'ALL_USERS'
                    }

                    batch.put_item(Item=item)
                except Exception as e:
                    print(f"❌ Skipping row due to error: {e}")

    print("✅ Data inserted successfully from Kaggle dataset!")

In [ ]:

def insert_data(csv_file):
    # Read the CSV file
    df = pd.read_csv(csv_file)
    
    # Process each row
    for _, row in df.iterrows():
        # Parse hashtags from string to list if needed
        hashtags = []
        if isinstance(row['hashtags'], str) and row['hashtags']:
            hashtags = row['hashtags'].strip('[]').replace("'", "").split(', ')
        
        # Basic item for the main table
        item = {
            'user_name': str(row['user_name']),
            'date': str(row['date']),
            'user_location': str(row['user_location']) if pd.notna(row['user_location']) else 'unknown',
            'user_description': str(row['user_description']) if pd.notna(row['user_description']) else '',
            'user_created': str(row['user_created']),
            'user_followers': int(row['user_followers']) if pd.notna(row['user_followers']) else 0,
            'user_friends': int(row['user_friends']) if pd.notna(row['user_friends']) else 0,
            'user_favourites': int(row['user_favourites']) if pd.notna(row['user_favorites']) else 0,
            'user_verified': bool(row['user_verified']) if pd.notna(row['user_verified']) else False,
            'text': str(row['text']),
            'hashtags': hashtags,
            'source': str(row['source']) if pd.notna(row['source']) else '',
            'is_retweet': bool(row['is_retweet']) if pd.notna(row['is_retweet']) else False,
            'dummy_attribute': 'ALL_USERS'  # For FollowersIndex
        }

        # Insert the main item
        table.put_item(Item=item)
        
        # For HashtagIndex, create an entry for each hashtag
        for tag in hashtags:
            hashtag_item = item.copy()
            hashtag_item['hashtag'] = tag
            # This would go to the HashtagIndex through GSI
            table.put_item(Item=hashtag_item)

## Query 1

In [19]:

# Query 1: All tweets of a user
def get_user_tweets(username):
    response = table.query(
        KeyConditionExpression=Key('user_name').eq(username)
    )
    return response['Items']


## Query 2

In [20]:

# Query 2: All tweets by the users from the same location
def get_tweets_by_location(location):
    response = table.query(
        IndexName='LocationDateIndex',
        KeyConditionExpression=Key('user_location').eq(location)
    )
    return response['Items']

## Query 3

In [21]:

# Query 3: Top k users with most followers
def get_top_users_by_followers(k=10):
    response = table.query(
        IndexName='FollowersIndex',
        KeyConditionExpression=Key('dummy_attribute').eq('ALL_USERS'),
        ScanIndexForward=False,  # Descending order (highest followers first)
        Limit=k
    )
    
    # Since there might be duplicate users (one per tweet), get unique users
    unique_users = {}
    for item in response['Items']:
        if item['user_name'] not in unique_users:
            unique_users[item['user_name']] = item
            
    # Return top k unique users
    return list(unique_users.values())[:k]


## Query 4

In [22]:
# Query 4: Tweets by top k users with the most followers
def get_tweets_by_top_users(k=10):
    # First get top k users
    top_users = get_top_users_by_followers(k)
    
    # Then get all tweets by these users
    all_tweets = []
    for user in top_users:
        username = user['user_name']
        user_tweets = get_user_tweets(username)
        all_tweets.extend(user_tweets)
    
    return all_tweets

## Query 5

In [23]:
# Query 5: Top k tweets with the most matching tags
def get_tweets_with_most_hashtags(k=10):
    # For this we need to scan the table and count hashtags
    # This is not efficient but DynamoDB doesn't support aggregation natively
    response = table.scan()
    items = response['Items']
    
    # Continue scanning if we have more items (pagination)
    while 'LastEvaluatedKey' in response:
        response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
        items.extend(response['Items'])
    
    # Sort by hashtag count
    items.sort(key=lambda x: len(x.get('hashtags', [])), reverse=True)
    
    return items[:k]

## Query 6

In [24]:
# Query 6: Delete all posts of user with followers less than a threshold
def delete_users_with_few_followers(threshold=100):
    # First find all users with followers below threshold
    # This requires a full scan which is expensive for large tables
    response = table.scan(
        FilterExpression=Attr('user_followers').lt(threshold)
    )
    items_to_delete = response['Items']
    
    # Continue scanning if we have more items
    while 'LastEvaluatedKey' in response:
        response = table.scan(
            FilterExpression=Attr('user_followers').lt(threshold),
            ExclusiveStartKey=response['LastEvaluatedKey']
        )
        items_to_delete.extend(response['Items'])
    
    # Delete each item
    deleted_count = 0
    for item in items_to_delete:
        table.delete_item(
            Key={
                'user_name': item['user_name'],
                'date': item['date']
            }
        )
        deleted_count += 1
    
    return {
        'deleted_count': deleted_count,
        'message': f"Deleted {deleted_count} tweets from users with fewer than {threshold} followers"
    }


## Running Insertion

In [ ]:
# Run the insertion
if __name__ == "__main__":
    try:
        # Create the table if it doesn't exist
        create_table()
        print("Table created successfully")
    except Exception as e:
        print(f"Table might already exist: {e}")
    


In [48]:
# AWS Configuration (add this at the top)
AWS_REGION = 'us-east-1'  # Change to your preferred region
AWS_PROFILE = 'default'   # Change if using a named profile
def configure_aws():
    """Configure AWS credentials and region"""
    try:
        # Option 1: Use AWS credentials file (~/.aws/credentials)
        session = boto3.Session(profile_name=AWS_PROFILE, region_name=AWS_REGION)
        
        # Verify credentials work
        sts = session.client('sts')
        sts.get_caller_identity()
        return session
    except (NoCredentialsError, ClientError) as e:
        print(f"AWS configuration error: {str(e)}")
        print("Please ensure you have:")
        print("1. Configured AWS credentials (aws configure)")
        print("2. Set a default region")
        print("3. Or set AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY, and AWS_REGION environment variables")
        raise


In [51]:
    # Insert data from CSV
insert_data("data/Bitcoin_tweets.csv")
print("Data inserted successfully")

❌ Error in row 28: Unable to locate credentials
Problematic values: {'user_name': 'FOMO King', 'date': '2021-02-10 23:42:49', 'user_followers': 168.0, 'user_friends': 822, 'user_favourites': 6919}
❌ Error in row 55: Unable to locate credentials
Problematic values: {'user_name': 'Max Payne Crypto', 'date': '2021-02-10 23:29:53', 'user_followers': 7.0, 'user_friends': 36, 'user_favourites': 23}
❌ Error in row 82: Unable to locate credentials
Problematic values: {'user_name': 'Ｆｉｎｔｅｃｈ ＵＫ 🇬🇧', 'date': '2021-02-10 23:18:15', 'user_followers': 1950.0, 'user_friends': 110, 'user_favourites': 1378}
❌ Error in row 113: Unable to locate credentials
Problematic values: {'user_name': 'yadio', 'date': '2021-02-10 23:02:02', 'user_followers': 3573.0, 'user_friends': 83, 'user_favourites': 122}
❌ Error in row 138: Unable to locate credentials
Problematic values: {'user_name': 'Free Bitcoin', 'date': '2021-02-10 22:53:26', 'user_followers': 33038.0, 'user_friends': 1024, 'user_favourites': 8348}
❌ Err

NoCredentialsError: Unable to locate credentials

## Running Query

In [25]:
# Example usage
if __name__ == "__main__":
    # Example of how to call these functions
    print("Top 5 users by followers:")
    top_users = get_top_users_by_followers(5)
    for user in top_users:
        print(f"{user['user_name']}: {user['user_followers']} followers")
    
    print("\nExample tweets from New York:")
    ny_tweets = get_tweets_by_location("New York")
    for tweet in ny_tweets[:3]:  # Just show first 3
        print(f"{tweet['user_name']} ({tweet['date']}): {tweet['text'][:100]}...")

Top 5 users by followers:


NoCredentialsError: Unable to locate credentials

In [ ]:

# Connect to local DynamoDB
dynamodb = boto3.resource(
    'dynamodb',
    region_name='us-east-1',
    endpoint_url='http://localhost:8000'
)

table_name = 'BitcoinTweets'